Importa biblioteca Pandas, chama de 'pd' e libera o acesso ao Google Drive

In [10]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import math

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Configura o pandas para mostrar todas as colunas do dataframe
pd.options.display.max_columns = None


Mounted at /content/drive


Indica o caminho, lê e manipula o arquivo (Prepara o DF).

In [2]:
caminho_para_pasta = '/content/drive/MyDrive/ICD/Colab/Alexandre'
caminho_para_arquivo = '/chuvas_C_00935056.csv'

# O arquivo apresentou erros de separador e codificação.
# Os dados começam na linha 12
# Não usa coluna como índice
df = pd.read_csv(f'{caminho_para_pasta}{caminho_para_arquivo}', sep=';', encoding="ISO-8859-1", skiprows = 12, index_col=False, decimal=',')
#print(df.columns)
# Muda o tipo da coluna 'Data' para datetime (string --> datetime)
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice
df.set_index(['Data','NivelConsistencia'], inplace=True)

# Exclui colunas desnecessárias do dataframe
listColumnsDrop = []
for i in range(73):
  if i > 41:
    listColumnsDrop += [i]
listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10]
df = df.drop(df.columns[listColumnsDrop], axis=1)

# Avalia se existe uma mesma data com diferentes níveis de consistência
# Classificar o dataframe por data
df.sort_values("Data", inplace=True)

# Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
df = df.reset_index()

# apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice
df.set_index(['Data','NivelConsistencia'], inplace=True)

# transforma as colunas de chuva em linhas com as respectivas datas
# Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
# Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
# Correção explicada durante o processo:

# usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
# São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

# Fiz uma cópia da minha data em formato de texto
df['Data1']=df['Data'].astype(str)

# Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
df['Data1'] = df['Data1'].str[:-2]

# Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
# (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)

# praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

# Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
# aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

# Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
# Mas aqui também forma datas como '1989-02-31'
df['Data1'] = df['Data1'] + df['Dia1']

# A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
# vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

# Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
df.dropna(inplace=True, subset=['Data1'])

# reorganiza as colunas
df = df[['Data', 'NivelConsistencia', 'Chuva']]

# Classificar o dataframe por data
df.sort_values("Data", inplace=True)

# A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
df.set_index(['Data'], inplace=True)

Visualização do DataFrame por data

In [3]:
periodo = df.loc['2000-12-01':'2000-12-31']
periodo.head(5)

,NivelConsistencia,Chuva
Data,,
2000-12-01,2,0.0
2000-12-02,2,0.0
2000-12-03,2,0.0
2000-12-04,2,0.0
2000-12-05,2,0.0


discretizações temporais com indicadores

In [9]:
# Resample do dataframe 'df' para obter estatísticas mensais da coluna 'Chuva'
df_mes1 = df.resample('M').agg({'Chuva': [('Soma', 'sum'), ('Média','mean'), ('Desvio Padrão','std'), ('Máxima','max'), ('Falhas(%)', lambda x: x.isna().sum())]})['Chuva']

# Resample do dataframe 'df' para contar a quantidade de valores igual a 2 na coluna 'NivelConsistencia' para cada mês
df_mes2 = df.resample('M').agg({'NivelConsistencia': [('Consistidos(%)', lambda x: (x == 2).sum())]})['NivelConsistencia']

# Mesclar os dataframes 'df_mes1' e 'df_mes2' com base nos índices (datas)
df_mes = df_mes1.merge(df_mes2, left_index=True, right_index=True)

# Calcular a porcentagem de falhas em relação ao número de dias de cada mês
df_mes['Falhas(%)'] = round(100 * df_mes['Falhas(%)'] / df_mes.index.day, 2)

# Calcular a porcentagem de valores consistentes em relação ao número de dias de cada mês
df_mes['Consistidos(%)'] = round(100 * df_mes['Consistidos(%)'] / df_mes.index.day, 2)

# Resample do dataframe 'df' para obter estatísticas anuais da coluna 'Chuva'
df_ano1 = df.resample('Y').agg({'Chuva': [('Soma', 'sum'), ('Média','mean'), ('Desvio Padrão','std'), ('Máxima','max'), ('Falhas(%)', lambda x: x.isna().sum())]})['Chuva']

# Resample do dataframe 'df' para contar a quantidade de valores igual a 2 na coluna 'NivelConsistencia' para cada ano
df_ano2 = df.resample('Y').agg({'NivelConsistencia': [('Consistidos(%)', lambda x: (x == 2).sum())]})['NivelConsistencia']

# Mesclar os dataframes 'df_ano1' e 'df_ano2' com base nos índices (datas)
df_ano = df_ano1.merge(df_ano2, left_index=True, right_index=True)

# Calcular a porcentagem de falhas em relação ao número de dias do ano (dayofyear)
df_ano['Falhas(%)'] = round(100 * df_ano['Falhas(%)'] / df_ano.index.dayofyear, 2)

# Calcular a porcentagem de valores consistentes em relação ao número de dias do ano (dayofyear)
df_ano['Consistidos(%)'] = round(100 * df_ano['Consistidos(%)'] / df_ano.index.dayofyear, 2)

Configuração do gráfico: entradas(dataframe, data inicial, data final); saída(gráfico)

In [7]:
def grafico(dataframe, a, b):
    # Cria uma nova figura do tipo Bar do Plotly
    fig = go.Figure(data=[
        # Adiciona um gráfico de barras no eixo x com base no intervalo de datas de 'a' a 'b'
        go.Bar(x=dataframe.loc[a:b].index, y=dataframe.loc[a:b]['Soma'],
               # Adiciona texto de dica ao passar o mouse sobre cada barra, exibindo a porcentagem de falhas
               hovertext=[str(x) + '%' for x in dataframe.loc[a:b]['Falhas(%)'].tolist()])
    ])

    # Define o modo de agrupamento das barras no gráfico
    fig.update_layout(barmode='group')

    # Mostra o gráfico
    fig.show()

Chama a função gráfico

In [8]:
mes = df_mes.loc[df_mes.index.month == 6]
grafico(df_ano,'1989-01-01','2022-12-31')